# Test Bubble Sort with NOPs (Pipeline Hazard Diagnosis)

This notebook tests a version of bubble sort with NOP instructions inserted after loads and before stores.

**If this version works but the original doesn't, it confirms pipeline hazards in your processor.**

In [ ]:
from pynq import Overlay, MMIO
import time
import random

# Configuration
ARRAY_SIZE = 32
MAGIC_NUMBER = 0xDEADBEEF
DATA_RAM_START = 0x1000
ARRAY_START_OFFSET = 0x40
STATUS_FLAG_OFFSET = 0x1000
MAX_TIMEOUT = 10
POLL_DELAY = 0.001

BITSTREAM = "design_1_wrapper.bit"

print("Loading Overlay...")
overlay = Overlay(BITSTREAM)

iram_info = overlay.mem_dict['axi_bram_ctrl_1']
dram_info = overlay.mem_dict['axi_bram_ctrl_0']
iram = MMIO(iram_info['phys_addr'], iram_info['addr_range'])
dram = MMIO(dram_info['phys_addr'], dram_info['addr_range'])

gpio_info = overlay.ip_dict['axi_gpio_0']
gpio = MMIO(gpio_info['phys_addr'], gpio_info['addr_range'])

def halt_core():
    gpio.write(0x0, 0x0)

def run_core():
    gpio.write(0x0, 0x1)

print("Hardware mapped.")

In [ ]:
# Load instructions WITH NOPs to eliminate hazards
print("Loading bubble sort WITH NOPS (hazard-safe version)...\n")

halt_core()

instructions_with_nops = [
    "00001537",  # lui a0, 0x1
    "04050513",  # addi a0, a0, 0x40
    "01f00293",  # addi t0, zero, 31
    "02028e63",  # beq t0, zero, done
    "01f00313",  # addi t1, zero, 31
    "00050593",  # addi a1, a0, 0
    "02030463",  # beq t1, zero, outer_continue
    "0005a383",  # lw t2, 0(a1)
    "00000013",  # nop
    "00000013",  # nop
    "0045ae03",  # lw t3, 4(a1)
    "00000013",  # nop
    "00000013",  # nop
    "007e2eb3",  # slt t4, t3, t2
    "00000013",  # nop
    "000e8663",  # beq t4, zero, no_swap
    "01c5a023",  # sw t3, 0(a1)
    "00000013",  # nop
    "0075a223",  # sw t2, 4(a1)
    "00000013",  # nop
    "00458593",  # addi a1, a1, 4
    "fff30313",  # addi t1, t1, -1
    "fc5ff06f",  # jal zero, inner_loop (adjusted offset)
    "fff28293",  # addi t0, t0, -1
    "fb9ff06f",  # jal zero, outer_loop (adjusted offset)
    "00002637",  # lui a2, 0x2
    "deadf6b7",  # lui a3, 0xDEADF
    "eef68693",  # addi a3, a3, -273
    "00d62023",  # sw a3, 0(a2)
    "ff1ff06f"   # jal zero, done
]

for i, instr_hex in enumerate(instructions_with_nops):
    iram.write(i * 4, int(instr_hex, 16))

print(f"Loaded {len(instructions_with_nops)} instructions (with NOPs)")

In [ ]:
# Generate test array (use same seed for comparison)
print("Generating test array...\n")
test_array = [59, 98, 15, -90, -47, 86, 37, 32, 6, 93, 67, -39, -48, -24, 12, 9, -16, -46, -7, 29, -9, 1, 83, -20, -71, 59, 81, 76, 13, 51, 7, 89]
golden_result = sorted(test_array)

print(f"Test array: {test_array}")
print(f"\nExpected: {golden_result}")

In [ ]:
# Inject array and reset
halt_core()

status_addr = DATA_RAM_START + STATUS_FLAG_OFFSET
array_base = DATA_RAM_START + ARRAY_START_OFFSET

dram.write(status_addr, 0x00000000)

for i, value in enumerate(test_array):
    dram.write(array_base + i * 4, value & 0xFFFFFFFF)

print("Array injected. Starting core...")
time.sleep(0.02)
run_core()
time.sleep(0.02)

In [ ]:
# Poll for completion
print("Waiting for completion...\n")
start_time = time.time()

while True:
    status = dram.read(status_addr)
    if status == MAGIC_NUMBER:
        elapsed = time.time() - start_time
        print(f"COMPLETED in {elapsed:.3f} seconds")
        break
    if time.time() - start_time > MAX_TIMEOUT:
        print("TIMEOUT!")
        break
    time.sleep(POLL_DELAY)

In [ ]:
# Read back and verify
result_array = []
for i in range(ARRAY_SIZE):
    val = dram.read(array_base + i * 4)
    signed_val = val if val < 0x80000000 else val - 0x100000000
    result_array.append(signed_val)

print("=" * 70)
print("RESULTS WITH NOPs")
print("=" * 70)
print(f"\nExpected: {golden_result}")
print(f"\nHardware: {result_array}")

mismatches = []
for i, (expected, actual) in enumerate(zip(golden_result, result_array)):
    if expected != actual:
        mismatches.append((i, expected, actual))

if len(mismatches) == 0:
    print("\n" + "=" * 70)
    print("✓✓✓ TEST PASSED WITH NOPs! ✓✓✓")
    print("=" * 70)
    print("\n⚠️  DIAGNOSIS: Your hazard unit is NOT detecting/handling:")
    print("   - Load-use hazards")
    print("   - Memory dependencies")
    print("\n   Fix your hazard_unit.v to properly stall the pipeline!")
else:
    print("\n" + "=" * 70)
    print("✗ TEST STILL FAILED (even with NOPs)")
    print("=" * 70)
    print(f"Mismatches: {len(mismatches)}")
    for idx, expected, actual in mismatches[:10]:
        print(f"  [{idx}]: Expected {expected}, Got {actual}")
    print("\n⚠️  DIAGNOSIS: Problem is more severe than simple hazards.")
    print("   Check: ALU operations, memory controller, forwarding paths")